In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
import basty.project.experiment_processing as experiment_processing
import seaborn as sns
sns.set_theme(style = 'dark')
from basty.utils import misc
from process_results import BehaviorData
from filter_flies import FilterData
from input import Input

import sys
sys.path.append("/home/grover/Desktop/gitDLC/basty")

In [2]:
CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"
project = experiment_processing.Project(
   CONFIG_PATH
)
FPS = project.fps
FIG_PATH = r'C:\Users\Grover\Documents\GitHub\deepsleepfigs'

RESULTS_FOLDER = r"Z:\mfk\basty-projects\backup-allnohaltere\results\semisupervised_pair_kNN\predictions.15NN.neighbor_weights-distance.neighbor_weights_norm-log_count.activation-standard.voting-soft.voting_weights-None\exports"

BODY_PART_SETS = {'ProboscisPumping':'prob'}

BEHAVIORS = [
'Idle&Other',
'PosturalAdjustment&Moving',
'Feeding',
'Grooming',
'ProboscisPumping']

In [3]:
io_process = Input(project, RESULTS_FOLDER)
expt_info_df = io_process.load_expt_info()
df_data = io_process.load_predictions()
df_data.drop(['HaltereSwitch','Noise'],axis=1,inplace=True)
io_process.create_binary_masks_subfolders(BEHAVIORS)

In [4]:
llh = misc.get_likelihood(project.data_path_dict,CONFIG_PATH,n_workers=None)

Loading existing llh_df.pickle file...


In [5]:
process_data = BehaviorData(df_data,binary_mask_threshold=0.8)

In [ ]:
process_data.process_expt_names_parallel(llh,io_process.get_binary_mask_subfolder('ProboscisPumping'))

In [ ]:
llh  = llh[llh['ExptNames'].isin(df_data.ExptNames.unique())]

In [ ]:
newDf = pd.merge(df_data,expt_info_df,left_on = 'ExptNames',right_on = 'ExptNames')

In [ ]:
# Create a folder named binary mask and a subfolder for each predicted behavior.
# Within that folder save binary mask generated using the likelihood score


In [ ]:
wt_all = newDf[newDf['SD']==False]
male_wt = wt_all[wt_all['Sex'] == 'M']
female_wt = wt_all[wt_all['Sex'] == 'F']

In [ ]:
data = BehaviorData(behaviors=BEHAVIORS)

In [ ]:
newDf

In [ ]:
newDf

In [ ]:
filterer = FilterData(newDf,llh)

In [ ]:
filterer.plot_mean_likelihood()

In [ ]:
binary_mask = filterer.create_binary_masks(llh, BODY_PART_SETS, 0.7)

In [ ]:
masked_data_df = filterer.apply_binary_masks(binary_mask,newDf)

In [ ]:
masked_data_df

In [ ]:
binary_mask

In [ ]:
masked_data_df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is called 'masked_data_df'
# and has columns 'ExptNames', 'ValueColumn' (replace with your actual column name)
# Add row numbers if they don't exist
masked_data_df['Row'] = masked_data_df.index

# Pivot the DataFrame
pivoted_df = masked_data_df.pivot(index='ExptNames', columns='Row', values='ProboscisPumping')

# Create a heatmap using Seaborn, handling NaN values with a different color (e.g., gray)
plt.figure(figsize=(16, 9))
sns.heatmap(pivoted_df, cmap='coolwarm', linewidths=0.5, cbar_kws={'ticks': [0, 1]}, mask=pivoted_df.isna(), linecolor='gray')

# Set plot attributes
plt.xlabel('Rows')
plt.ylabel('ExptNames')
plt.title('Heatmap of Data')

# Save the plot as a PNG file
FIG_PATH = r'C:\Users\Grover\Documents\GitHub\deepsleepfigs'
plt.savefig(f"{FIG_PATH}\\heatmap.png", dpi=300, bbox_inches='tight')

# Display the plot in the notebook (optional)
plt.show()



In [ ]:
plot_folder = r"C:\Users\Grover\Documents\GitHub\deepsleepfigs\score_plots"
if not os.path.isdir(plot_folder):
    os.makedirs(plot_folder)

In [ ]:
male_wt['Time'] = male_wt['Idx'].apply(BehaviorData.get_time_stamp)
male_wt.index = pd.to_datetime(list(male_wt["Time"]))
rate = '1S'
male_wt_rs = BehaviorData.resample_df(male_wt,rate,BEHAVIORS)
male_wt_rs['TimeStamp'] = male_wt_rs.index

In [ ]:
male_wt

In [ ]:
llh['Time'] = llh['Idx'].apply(BehaviorData.get_time_stamp)

In [ ]:
BehaviorData.pivot_and_plot(male_wt_rs,'mal_b_scr_',plot_folder,rate,BEHAVIORS)

In [ ]:
import os

In [ ]:
os.listdir(plot_folder)

In [ ]:
df_pivoted = male_wt_rs.pivot(index='ExptNames', columns='TimeStamp', values='ProboscisPumping')
a4_dims = (25.7, 5.27)
fig, ax = plt.subplots(figsize=a4_dims)
plt.title('ProboscisPumping')
ax = sns.heatmap(df_pivoted, cmap='YlGnBu')

In [ ]:
male_wt_rs

In [ ]:
result_df_parallel = BehaviorData.process_expt_names_parallel(male_wt, llh)
print(result_df_parallel)

In [ ]:

unique_expt_names = male_wt['ExptNames'].unique()

unmasked_data = np.vstack([result_df_parallel[f'{expt_name}_unmasked'] for expt_name in unique_expt_names])
masked_data = np.vstack([result_df_parallel[f'{expt_name}_masked'] for expt_name in unique_expt_names])

# Plot the heatmap
plt.figure(figsize=(10, 3))
sns.heatmap(data, cmap='YlGnBu')
plt.xlabel('Data')
plt.ylabel('Arrays')
plt.yticks([0.5, 1.5], ['Array 1', 'Array 2'], rotation=0)
plt.show()


In [ ]:
unique_expt_names = male_wt['ExptNames'].unique()

a4_dims = (25.7, 5.27)
# Plot the unmasked data heatmap
#plt.figure(figsize=(a4_dims))
#sns.heatmap(unmasked_data, cmap='YlGnBu', yticklabels=unique_expt_names)
#plt.xlabel('Data')
#plt.ylabel('ExptNames')
#plt.title('Unmasked Data')
#plt.savefig(os.path.join(plot_folder,'unmasked_data_heatmap_full_07.png'), dpi=300)  # Save the figure as a PNG file
#plt.show()



In [ ]:

# Plot the masked data heatmap
plt.figure(figsize=(a4_dims))
sns.heatmap(masked_data, cmap='YlGnBu', yticklabels=unique_expt_names)
plt.xlabel('Data')
plt.ylabel('ExptNames')
plt.title('Masked Data')
plt.savefig(os.path.join(plot_folder,'masked_data_heatmap_full_09.png'), dpi=300)
plt.show()

In [ ]:
snap_paths = [{expt_names : os.path.join(project.project_path,expt_names,'snap_stft.pkl')} for expt_names in project.expt_names]
snap_col_names = [os.path.join(project.project_path,expt_names,'ftname_to_snapft.yaml') for expt_names in project.expt_names]

In [ ]:
df_data

In [ ]:
df_data